In [1]:
import csv
from gmplot import gmplot
import json
from pprint import pprint
import urllib.request  as urllib2 
import os
import time
import math
import random

key = "4BGs4umzWE9Bg7fluGQR7gToO6Qlageu"
EARTH_POLAR_RADIUS_METER = 6356800
EARTH_EQUATORIAL_RADIUS_METER = 6378100
meterWidth = 1000
count = 50

def queryEventfulWithId(id):
	url = "http://api.eventful.com/json/events/get?app_key=srG2DMrq4VpRxGvw&id="+id
	response = urllib2.urlopen(url).read()
	rspJSON = json.loads(response)
	return rspJSON
    
# Gets the metric length corresponding to the latitude, in degrees.
def getLatitudeWidth(meterWidth):
    return meterWidth/(2*math.pi*EARTH_POLAR_RADIUS_METER ) *360

# Convert the metric width to the corresponding longitude width at latitude
def getLongitudeWidthAtLatitue(latitude, meterWidth):
    circleMeterLengthAtLatitude = 2 * math.pi * EARTH_EQUATORIAL_RADIUS_METER * math.cos(latitude / 180 * math.pi)
    return meterWidth/circleMeterLengthAtLatitude * 360

def degreesToRadians(degrees):
	return degrees * math.pi / 180

def distanceInMetersBetweenEarthCoordinates(lat1, lon1, lat2, lon2):
	earthRadiusKm = 6371
	dLat = degreesToRadians(lat2-lat1)
	dLon = degreesToRadians(lon2-lon1)
	lat1 = degreesToRadians(lat1)
	lat2 = degreesToRadians(lat2)
	a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2) 
	c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a)) 
	return earthRadiusKm * c * 1000


# Generates a uniform distribution of count coordinates
# Boundary defined by lat,lon of center corner and width in meters
def generateCoordsWithinSquare(centerLat,centerLon,meterWidth,count,gmap,map_file):

	# Generate rectangle width in latitude and longitude coordinates
	dLat = getLatitudeWidth(meterWidth)
	dLon = getLongitudeWidthAtLatitue(centerLat,meterWidth)

	# Generate and show rectangle boundaries
	upperLeftLat = centerLat + (dLat / 2.0)
	upperLeftLon = centerLon - (dLon / 2.0)

	lowerRightLat = upperLeftLat - dLat
	lowerRightLon = upperLeftLon + dLon

	upperRightLat = upperLeftLat
	upperRightLon = lowerRightLon

	lowerLeftLat = lowerRightLat
	lowerLeftLon = upperLeftLon

	latitudes = []
	longitudes = []
	distances_to_center = []

	# Generate count randomly distributed coordinates
	for i in range(count):
		lat = random.uniform(upperLeftLat, lowerLeftLat)
		lon = random.uniform(upperLeftLon, upperRightLon)
		dist = distanceInMetersBetweenEarthCoordinates(centerLat,centerLon,lat,lon)
		latitudes.append(lat)
		longitudes.append(lon)
		distances_to_center.append(dist)
        
	boundaries = [[upperLeftLat, upperRightLat, lowerRightLat, lowerLeftLat, upperLeftLat], [upperLeftLon, upperRightLon, lowerRightLon, lowerLeftLon, upperLeftLon]]

	return latitudes,longitudes,distances_to_center

def getSpeedAtLoc(lat,lon):
    try:
        url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key="+key+"&point="+lat+","+lon+"&unit=MPH"
        response = urllib2.urlopen(url).read()
        rspJSON = json.loads(response)
        data = rspJSON['flowSegmentData']
        freeFlowSpeed = data["freeFlowSpeed"]
        currentSpeed = data["currentSpeed"]
        confidence = data["confidence"]
        ratio = round(currentSpeed/float(freeFlowSpeed),2)

        #TODO: put average of returned segments lat and lon here

        return [str(lat),str(lon),str(freeFlowSpeed),str(currentSpeed),str(ratio),str(confidence),str(time.time())]
    except urllib2.HTTPError as err:
        print ("not found")



import pandas as pd
columns = ['event','title','venue','venue_lat','venue_lon','sample_lats','sample_lons','sample_dist']
df = pd.DataFrame(columns=columns)



# Put file name below

In [2]:

df = pd.read_csv("March23_Pacific Design Center.csv")

### When you print the block, it prints a string of a list, copy just the list part, without the string quotes, in lats

In [3]:
df['sample_lats'][0]

'[34.08586061445081, 34.08028377269614, 34.07843903344922, 34.07857638331549, 34.084458722822134, 34.08113713902942, 34.08273460517626, 34.081485120423864, 34.083561357775345, 34.08481543606941, 34.08369075606531, 34.0829693202046, 34.080362434654326, 34.080952077066996, 34.08158013534032, 34.078828113209234, 34.077709781224584, 34.0862374993732, 34.08074077327185, 34.07857701940844, 34.082873859362465, 34.0786753272318, 34.082080807171344, 34.078039787277724, 34.08133782345184, 34.08077330177384, 34.0808288794761, 34.08179623032275, 34.08264228447595, 34.08030882938809, 34.07985699715273, 34.0791751423791, 34.07847429784208, 34.085030392807276, 34.079744716045006, 34.08652190175058, 34.080679823607916, 34.07890377284466, 34.079521804764, 34.08003391046039, 34.0834283791161, 34.08478084621665, 34.07796852593396, 34.0833833826394, 34.08007458295607, 34.08260817739367, 34.08159705270265, 34.079238696052364, 34.08548819393675, 34.086147518057054]'

In [4]:
lats = [34.08586061445081, 34.08028377269614, 34.07843903344922, 34.07857638331549, 34.084458722822134, 34.08113713902942, 34.08273460517626, 34.081485120423864, 34.083561357775345, 34.08481543606941, 34.08369075606531, 34.0829693202046, 34.080362434654326, 34.080952077066996, 34.08158013534032, 34.078828113209234, 34.077709781224584, 34.0862374993732, 34.08074077327185, 34.07857701940844, 34.082873859362465, 34.0786753272318, 34.082080807171344, 34.078039787277724, 34.08133782345184, 34.08077330177384, 34.0808288794761, 34.08179623032275, 34.08264228447595, 34.08030882938809, 34.07985699715273, 34.0791751423791, 34.07847429784208, 34.085030392807276, 34.079744716045006, 34.08652190175058, 34.080679823607916, 34.07890377284466, 34.079521804764, 34.08003391046039, 34.0834283791161, 34.08478084621665, 34.07796852593396, 34.0833833826394, 34.08007458295607, 34.08260817739367, 34.08159705270265, 34.079238696052364, 34.08548819393675, 34.086147518057054]

### When you print the block, it prints a string of a list, copy just the list part, without the string quotes, in lons

In [5]:
df['sample_lons'][0]

'[-118.3798088663451, -118.38324408331094, -118.37825909718558, -118.380796034588, -118.3849543701942, -118.38409629673254, -118.38504626407496, -118.38392955791946, -118.37904177113609, -118.37862235278986, -118.38150156328132, -118.38254720363219, -118.38426355893957, -118.38473311094548, -118.37718545283292, -118.38683501127548, -118.38067574956466, -118.38616973946333, -118.3837462200013, -118.38580338575497, -118.38544875297657, -118.38527540573952, -118.38565690891967, -118.38745303948774, -118.37711559590498, -118.37678607775503, -118.38447111374829, -118.38687800354114, -118.38366140325735, -118.37708429373431, -118.38034674344014, -118.38269226728544, -118.38426409761011, -118.38115768049109, -118.3823634461586, -118.38173422893168, -118.37817359371556, -118.38433460166644, -118.37806763711056, -118.38742481892825, -118.38024238977339, -118.37684277998589, -118.3822017743235, -118.37692482353911, -118.38504233811133, -118.38127112718722, -118.38174752413778, -118.3826302160222

In [ ]:
lons = [-118.3798088663451, -118.38324408331094, -118.37825909718558, -118.380796034588, -118.3849543701942, -118.38409629673254, -118.38504626407496, -118.38392955791946, -118.37904177113609, -118.37862235278986, -118.38150156328132, -118.38254720363219, -118.38426355893957, -118.38473311094548, -118.37718545283292, -118.38683501127548, -118.38067574956466, -118.38616973946333, -118.3837462200013, -118.38580338575497, -118.38544875297657, -118.38527540573952, -118.38565690891967, -118.38745303948774, -118.37711559590498, -118.37678607775503, -118.38447111374829, -118.38687800354114, -118.38366140325735, -118.37708429373431, -118.38034674344014, -118.38269226728544, -118.38426409761011, -118.38115768049109, -118.3823634461586, -118.38173422893168, -118.37817359371556, -118.38433460166644, -118.37806763711056, -118.38742481892825, -118.38024238977339, -118.37684277998589, -118.3822017743235, -118.37692482353911, -118.38504233811133, -118.38127112718722, -118.38174752413778, -118.38263021602229, -118.38051305240383, -118.38011543038105]

### When you print the block, it prints a string of a list, copy just the list part, without the string quotes, in sampledistance

In [6]:
df['sample_dist'][0]

'[457.68014299453085, 238.49061656217046, 543.2345470069916, 417.43404195022265, 368.35632279667755, 221.91925938962106, 283.0325737354771, 190.1670814560203, 316.1412807712879, 430.04021573058054, 174.7655466988815, 98.69577779451235, 287.74907399835706, 282.94956281485355, 452.6151121001456, 577.8406159992596, 513.362831731017, 589.3345924698399, 224.32636153809293, 529.8416583146127, 322.6471108382339, 490.7006426386275, 332.7963976038435, 678.5863517896618, 463.6937913825943, 509.17386803164646, 269.52139062994934, 447.14734766084536, 157.26407176161993, 502.2838982329475, 302.46682607697494, 339.94020171816413, 460.4320735709326, 326.7871479137799, 272.9225639267249, 483.09271644564416, 393.912719582038, 421.3844371438924, 471.0698492301301, 550.1365859798938, 216.07887768711274, 559.3663720232468, 469.07137930085685, 490.0528067394413, 362.27641338099147, 85.44103682037615, 70.90276186617778, 332.0103288369142, 393.4786470874674, 475.11333813268806]'

In [ ]:
sampledistance = [457.68014299453085, 238.49061656217046, 543.2345470069916, 417.43404195022265, 368.35632279667755, 221.91925938962106, 283.0325737354771, 190.1670814560203, 316.1412807712879, 430.04021573058054, 174.7655466988815, 98.69577779451235, 287.74907399835706, 282.94956281485355, 452.6151121001456, 577.8406159992596, 513.362831731017, 589.3345924698399, 224.32636153809293, 529.8416583146127, 322.6471108382339, 490.7006426386275, 332.7963976038435, 678.5863517896618, 463.6937913825943, 509.17386803164646, 269.52139062994934, 447.14734766084536, 157.26407176161993, 502.2838982329475, 302.46682607697494, 339.94020171816413, 460.4320735709326, 326.7871479137799, 272.9225639267249, 483.09271644564416, 393.912719582038, 421.3844371438924, 471.0698492301301, 550.1365859798938, 216.07887768711274, 559.3663720232468, 469.07137930085685, 490.0528067394413, 362.27641338099147, 85.44103682037615, 70.90276186617778, 332.0103288369142, 393.4786470874674, 475.11333813268806]

### In the code below, just change the EDIT ME part

In [ ]:
counter = 0
while counter <= 11:
    columns1 = ['eventname','loc_type','lat','lon','freeflowspeed','currentspeed','ratio','confidence','CST Time','dist_from_center']
    i = 0
    dfadd = pd.DataFrame([['ignore this row',"2",1.1,1.1,21,21,1,1,1,0]],columns=columns1)

    loc_type = "C"
    venue_name = str(df['venue'][0])
    lat = df['venue_lat'][i]
    lon = df['venue_lon'][i]
    lat,lon,freeFlowSpeed,currentSpeed,ratio,confidence,timeticks = getSpeedAtLoc(str(lat),str(lon))
    dfadd.loc[len(dfadd)]= [str(df['title'][i]),loc_type,lat,lon,freeFlowSpeed,currentSpeed,ratio,confidence,time.ctime(int(float((timeticks)))),0]

    for j in range(50):
        loc_type = "S"
        lat,lon,freeFlowSpeed,currentSpeed,ratio,confidence,timeticks = getSpeedAtLoc(str(lats[j]),str(lons[j]))
        dfadd.loc[len(dfadd)] = [str(df['title']),loc_type,lat,lon,freeFlowSpeed,currentSpeed,ratio,confidence,time.ctime(int(float((timeticks)))),sampledistance[j]]

    ######EDIT ME##########
    
    # In the string belong change the date from March24 to whatever date it is.    
    
    dfadd.to_csv('March30_' + venue_name + str(time.ctime(int(float((timeticks))))) + '.csv')
    if counter == 11:
        break
    time.sleep(1800)
    counter += 1